In [ ]:
top_path = r''

## for running the second round of percentile based cleaning
#os.makedirs(os.path.join(TOP_PATH, 'round 2')
#SAVE_PATH = os.path.join(TOP_PATH, 'round 2')
#IMGS = tuple(np.load(os.path.join(TOP_PATH, 'INDEXED test.npy'), allow_pickle=True))
#NAMES = tuple(np.load(os.path.join(TOP_PATH, 'NAMES test.npy'), allow_pickle=True))
#print(len(IMGS), len(NAMES))

str_filter = 'orientation'
file_type = '.tif'
process_type = 'float'

str_trim = len(file_type)
files = os.listdir(top_path)

arrs = []
names = []
for i in files:
    if i.endswith(str_filter):
        names.append(i[:-str_trim])
        if 'retardance' in i:
            arrs.append(imread(os.path.join(top_path,i)))
        elif 'orientation' in i:
            if process_type == 'float':
                arrs.append(np.around((io.imread(os.path.join(TOP_PATH,i))/100).astype(np.float), decimals=3))
            elif process_type == 'int':
                arrs.append((io.imread(os.path.join(TOP_PATH,i))/100).astype(np.uint8))
            else:
                print('preprocessing info incomplete')
        else:
            print('unknown image type')
    else:
        continue
names = tuple(names)
arrs = tuple(arrs)

for i in arrs:
    plt.imshow(i, cmap = 'hsv')
    plt.show() 

##--------------------------------------------------------------------------------------
    
percentiles, pairs, filt_imgs, px_ct, ob_ct, l_imgs = [], [], [], [], [], []
for i in arrs:
    t = tuple(np.percentile(i, np.arange(0,101,5)).astype(float)) #based on frequency of angles in image
    #t = tuple(np.arange(0,181,2).astype(int)) #fixed angle bin width
    #t = tuple(np.arange(0,np.amax(i),10).astype(int)) #fixed angle bin width retardance
    percentiles.append(t)
    p = tuple([(t[j], t[j+1]) for j in range(len(t)-1)])
    paris.append(p)
    f_i = []
    px = []
    ob = []
    l_i = []
    for j in p:
        a = np.copy(i)
        a[i <= j[0]] = 0
        a[i > j[1]] = 0
        f_i.append(a)
        l = label(binary_opening(a, structure=square(3)), structure=square(3))
        l_i.append(l[0])
        ob.append(l[1])
        px.append(np.sum(a.astype(bool).astype(int)))
    filt_imgs.append(tuple(f_i))
    px_ct.append(tuple(px))
    ob_ct.append(tuple(ob))
    l_imgs.append(tuple(l_i))
percentiles, pairs, filt_imgs, px_ct, ob_ct, l_imgs = (tuple(percentiles), 
                                                       tuple(pairs), tuple(filt_imgs), 
                                                       tuple(px_ct), 
                                                       tuple(ob_ct), 
                                                       tuple(l_imgs))

##--------------------------------------------------------------------------------------

sizes, masks, indexed, tissues = [], [], [], []
for i in range(len(l_imgs)):
    i_img = l_imgs[i]
    
    stack = np.sum(np.stack(i_img), axis = 0)
    masks.append(stack.astype(np.uint8))
    a = np.copy(imgs[i])
    a[stack == False] = 0
    indexed.append(a)
    tissues.append(label(a, structure=square(3)))
    
    s1 = []
    for j in range(i_img):
        j_img = i_img[j]
        ids = np.arange(1, obj_ct[i][j]+1, 1)
        s2 = []
        for k in ids:
            k_size = np.where(j_img.flatten() == k)[0].shape[0]
            s2.append(k_size)
        s2 = np.concatenate((np.expand_dims(ids, axis = 1), np.expand_dims(s2, axis = 1)), axis = 1)
        s1.append(s2)
    sizes.append(tuple(s1))
sizes, masks, indexed, tissues = (tuple(sizes), tuple(masks), tuple(indexed), tuple(tissues))

np.save(os.path.join(SAVE_PATH, 'names.npy'), NAMES, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'imgs.npy'), IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'percentiles.npy'), PERCENTILES, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'pairs.npy'), PAIRS, allow_pickle=True)

np.save(os.path.join(SAVE_PATH, 'filtered images.npy'), FILT_IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'pixel count.npy'), PX_CT, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'object count.npy'), OB_CT, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'labeled images.npy'), L_IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'object sizes.npy'), SIZES, allow_pickle=True)

np.save(os.path.join(SAVE_PATH, 'masks.npy'), MASKS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'indexed.npy'), INDEXED, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'tissues.npy'), TISSUES, allow_pickle=True)